In [1]:
import sys
sys.path.insert(0, '../')

import numpy as np
import theano
import theano.tensor as T
import time

import lasagne as L

from HRED import HRED
from data_load.mt_load import load_mt, get_mt_voc, get_w2v_embs
from data_load.subtle_load import load_subtle
from diverse_beam_search import diverse_beam_search

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 780 (CNMeM is enabled with initial size: 80.0% of memory, cuDNN 4007)


In [2]:
# remember, now the pad value is the same as the <utt_end> token

pad_value = -1 # <utt_end>'s vector is the last one

In [3]:
t0 = time.time()
subtle_path = "/pio/data/data/mtriples/"

train_subtle = load_subtle(subtle_path, split=True, trim=200)
print time.time() - t0

IOError: Failed to interpret file '/pio/data/data/mtriples/Subtle_Dataset.triples.pkl' as a pickle

In [3]:
mt_path = "/pio/data/data/mtriples/"
# mt_path = "../DATA/MovieTriples_Dataset/"

train, valid, test = load_mt(path=mt_path, split=True, trim=200)
idx_to_w, w_to_idx, voc_size, freqs = get_mt_voc(path=mt_path, train_len=len(train))

In [4]:
word2vec_embs, word2vec_embs_mask = get_w2v_embs(path=mt_path)

w2v_train_mask = np.where(word2vec_embs_mask[:,0] == 1)[0]

In [5]:
%%time
hred_net = HRED(voc_size=voc_size,
                emb_size=300,
                lv1_rec_size=300, 
                lv2_rec_size=300, 
                out_emb_size=300, 
                num_sampled=200,
                ssoft_probs=freqs,
                emb_init=word2vec_embs)

Building the model...
Compiling theano functions...
Building a network for generating...
Done
CPU times: user 38.8 s, sys: 1.27 s, total: 40.1 s
Wall time: 1min 14s


In [6]:
hred_net.load_params('trained_models/subtleFixed_300_300_300_300_ssoft200unigr_bs30_cut200_early5.npz')

In [7]:
def print_utt(utt):
    return ' '.join([idx_to_w[x] for x in utt])

def rnd_next_word(probs, size=1):
    return np.random.choice(np.append(np.arange(probs.shape[0]-1), -1).astype(np.int32), 
                            size=size, p=probs)

def utt_to_array(utt):
    arr = np.array([w_to_idx.get(w, w_to_idx['<unk>']) for w in utt])[np.newaxis].astype(np.int32)
    arr[arr == -voc_size] = -1
    return arr

def context_summary(context, lookup=True):
    con_init = np.zeros((1, hred_net.lv2_rec_size), dtype=np.float32)
    for utt in context:
        con_init = hred_net.get_new_con_init_fn(utt_to_array(utt) if lookup else utt, con_init)
    return con_init

In [9]:
context = map(lambda x: np.array(x, dtype=np.int32)[np.newaxis], train[99:101])
lookup = False
for u in context:
    print print_utt(u[0])

<s> i couldn ' t say . </s>
<s> you were a prosecutor . </s>


In [8]:
context = ['<s> hi . </s>'.split(), '<s> hello , what \' s up ? </s>'.split()]
lookup = True

In [14]:
context = ['<s> yeah , okay . </s>'.split(), '<s> well , i guess i \' ll be going now . </s>'.split()]
lookup = True

In [17]:
context= ['<s> what would the table think about if it could think ? </s>'.split()]
lookup = True

In [19]:
context= ['<s> i saw a pretty good movie yesterday . </s>'.split()]
lookup = True

In [31]:
context= ['<s> hi . </s>'.split()]
lookup = True

In [20]:
beam_size = 20
group_size = 2
con_init = context_summary(context, lookup)
W = L.layers.get_all_param_values(hred_net.train_net)[31]
b = L.layers.get_all_param_values(hred_net.train_net)[32]
dec_init = np.repeat(np.tanh(con_init.dot(W) + b), beam_size, axis=0)

mean = True

beamsearch = diverse_beam_search(beam_size, group_size, dec_init, voc_size, hred_net, 
                                 init_seq=utt_to_array('<s> '.split()), rank_penalty=0, group_diversity_penalty=1, 
                                 seq_diversity_penalty=1, verbose_log=False)

# print print_utt(beamsearch)

len_bonus = lambda size: 0#np.log(size)**2

def fn_score(x, y, mean=mean, len_bonus=len_bonus):
    denom = (x.size - 1) if mean else 1
    return (y + len_bonus(x.size)) / denom

sort1 = sorted(beamsearch, key=lambda (x,y): fn_score(x, y), reverse=True)
sort2 = sorted(beamsearch, key=lambda x: ' '.join(print_utt(x[0][1:-1])))

for utt, scr in sort1:
    print '{:.3f}  '.format(fn_score(utt, scr)), print_utt(utt[1:-1])
    print ''

-1.234   is that what you ' re talking about ?

-1.262   what are you doing ?

-1.290   is that what you ' re doing ?

-1.371   what are you talking about ?

-1.399   so what ' s the matter ?

-1.478   it ' s not like that .

-1.493   is that what you think ?

-1.495   really ? well , i ' m sorry .

-1.510   yes , i ' m sorry .

-1.511   so what ' s your name ?

-1.576   is that what you ' re talking about ? <continued_utterance> i ' m sorry .

-1.581   i ' m sorry .

-1.586   oh , yeah .

-1.591   really ?

-1.619   are you kidding ?

-1.638   what are you talking about ? i ' m not gon na do that .

-1.641   did you see that ?

-1.655   what are you talking about ? i ' m not gon na do it .

-1.661   i ' m sorry , but it ' s okay .

-1.677   uh-huh .

-1.682   are you kidding ? <continued_utterance> i ' m sorry .

-1.687   is that what you ' re talking about ? <continued_utterance> i ' m sorry . i ' m not going to tell you anything . i don ' t know why you ' re here .

-1.688   is that

In [9]:
beam_size = 40
group_size = 2

for i in xrange(0, 100, 3):
    context = map(lambda x: np.array(x, dtype=np.int32)[np.newaxis], test[i:i+2])
    lookup = False
    for u in context:
        print print_utt(u[0])
        
    con_init = context_summary(context, lookup)
    W = L.layers.get_all_param_values(hred_net.train_net)[31]
    b = L.layers.get_all_param_values(hred_net.train_net)[32]
    dec_init = np.repeat(np.tanh(con_init.dot(W) + b), beam_size, axis=0)

    beamsearch = diverse_beam_search(beam_size, group_size, dec_init, voc_size, hred_net,
                                     init_seq=utt_to_array('<s> '.split()), )
    
    len_bonus = lambda size: 0#np.log(size)**2

    mean = True
        
    def fn_score(x, y, mean=mean, len_bonus=len_bonus):
        denom = (x.size - 1) if mean else 1
        return (y + len_bonus(x.size)) / denom

    for utt, scr in sorted(beamsearch, key=lambda (x,y): fn_score(x, y), reverse=True)[:1]:
        print '{:.3f}  '.format(fn_score(utt, scr)), print_utt(utt[1:-1])
        print ''
        
# for utt in beamsearch:
#     print_utt(utt)
#     print ''

<s> you ' re not going to let me die , are you ! ? <person> ' t let me die ! ignoring the <unk> </s>
<s> when we start running suck up your breath . you ' re going to get the wind knocked out of you . </s>
-1.285   what do you want me to do ?

<s> <person> says you have to be like me . </s>
<s> he does . </s>
-1.357   i don ' t think so .

<s> <person> . what ' s your <number> ? </s>
<s> reservation . </s>
-1.518   where ' d you go ?

<s> what the hell ' s going on here ? </s>
<s> we ' re in love with your roommate . </s>
-1.479   where are you going ?

<s> i ' m going to the market . i ' ll be back in a bit . </s>
<s> can i go with you ? </s>
-1.386   i don ' t want to go home .

<s> yes - </s>
<s> <person> ! <person> these men ! escort them to the <unk> ! </s>
-1.229   yes , sir .

<s> sure . </s>
<s> you can let me down now . </s>
-1.458   why don ' t you just go home ?

<s> i don ' t know . </s>
<s> we got a <unk> , you know . <person> -- did you hear what i said . ? </s>
-0.838   

In [10]:
hred_net.train_one_epoch(train_subtle2, 60)

Done 10 batches in 0.86s	training loss:	7.176166
Done 20 batches in 1.69s	training loss:	6.175796
Done 30 batches in 2.44s	training loss:	5.697121
Done 40 batches in 3.47s	training loss:	5.374195
Done 50 batches in 4.27s	training loss:	5.207509
Done 60 batches in 5.11s	training loss:	5.063584
Done 70 batches in 5.92s	training loss:	4.934647
Done 80 batches in 6.69s	training loss:	4.840843


KeyboardInterrupt: 

In [7]:
hred_net.validate(test, 30)

Done 100 batches in 4.07s
Done 200 batches in 8.39s
Done 300 batches in 12.95s
Done 400 batches in 17.68s
Done 500 batches in 21.70s
Done 600 batches in 26.34s
Done 700 batches in 31.03s
Done 800 batches in 35.56s
Done 900 batches in 40.01s
Done 1000 batches in 44.49s
Done 1100 batches in 48.81s
Done 1200 batches in 53.21s
Done 1300 batches in 57.49s
Done 1400 batches in 61.62s
Done 1500 batches in 66.13s
Done 1600 batches in 70.65s
Done 1700 batches in 74.90s
Done 1800 batches in 79.24s
Done 1900 batches in 83.80s
Done 2000 batches in 88.35s
Done 2100 batches in 92.88s
Done 2200 batches in 97.45s
Done 2300 batches in 101.66s
Done 2400 batches in 106.09s


3.2819798801888118

In [8]:
'''full softmax, bs=30'''
# train, 1 dir, 1 epoch: 3.485554076321884
# val: 3.455356876018342

# train, 2 dir, concat, 1 epoch: 3.4864403798772239
# val: 3.4579001751897063

# train, 2 dir, L2 + concat, 1 epoch: 3.4881669768474675
# val: 3.4584704095551695
# training time: ~4700s

'''sampled softmax'''
# bs=30
# train, 2 dir, L2 + concat, 1 epoch: 3.486180601246621
# val: 3.4811877499289308
# training time: ~2300s

# bs=60
# train, 2 dir, L2 + concat, 1 epoch: 3.5235153449672456
# val: 3.5063306987542759
# training time: ~1900s